### Import

In [ ]:
import pandas as pd
from PIL import Image
from io import BytesIO
import requests
import numpy as np

from skimage.io import imshow, imread
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from skimage.morphology import closing
from skimage.measure import label, regionprops, regionprops_table
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from tqdm import tqdm
import os

import cv2
from google.colab.patches import cv2_imshow
import tensorflow as tf
from keras.applications.vgg19 import preprocess_input

### Load Dataset

In [ ]:
df = pd.read_csv('all_combined_460.csv')

In [ ]:
# Initialize lists to store images and labels
images = []
labels = []

for index, row in df.iterrows():
    image_url = row['image_url']
    label = row['scientific_name']  # Adjust the column name according to your CSV

    response = requests.get(image_url)

    if response.status_code == 200:
        # Load the image using PIL
        img = Image.open(BytesIO(response.content))
        # Preprocess the image (e.g., resizing, normalization, etc.)
        img = img.resize((224, 224))  # Adjust the size as needed
        img = np.array(img) / 255.0  # Normalize pixel values
        images.append(img)
        labels.append(label)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

### Preprocessing Steps

In [ ]:
from skimage import exposure, img_as_ubyte

def preprocess_image(img):
    # Normalize the image
    img_normalized = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Convert to grayscale
    img_gray = cv2.cvtColor(img_normalized, cv2.COLOR_BGR2GRAY)

    # Apply Bilateral filter to reduce noise
    img_blurred = cv2.bilateralFilter(img_gray, d=9, sigmaColor=75, sigmaSpace=75)

    return img_gray, img_blurred

In [ ]:
def preprocess_image_NDI(img):
    # Normalize the image
    img_normalized = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Apply Bilateral filter to reduce noise
    img_blurred = cv2.bilateralFilter(img_normalized, d=9, sigmaColor=75, sigmaSpace=75)

    # Calculate NDI
    green_channel = img_blurred[:, :, 1].astype(np.float32)
    red_channel = img_blurred[:, :, 2].astype(np.float32)

    with np.errstate(divide='ignore', invalid='ignore'):
        ndi = np.true_divide(green_channel - red_channel, green_channel + red_channel)
        ndi[~np.isfinite(ndi)] = 0

    # Scale NDI to 0-255 range
    img_ndi = ((ndi + 1) * 127.5).astype(np.uint8)

    # Replicate the single channel to create a 3-channel image
    img_ndi_bgr = cv2.merge((img_ndi, img_ndi, img_ndi))

    # Convert NDI image to grayscale for visualization
    img_gray = cv2.cvtColor(img_ndi_bgr, cv2.COLOR_BGR2GRAY)

    return img_normalized, img_gray


In [ ]:
def contrast_enhancement(img_blurred, method='clahe', limit='0.01'):
    if method == 'clahe':
        # Apply CLAHE for contrast enhancement
        clahe = cv2.createCLAHE(clipLimit=limit, tileGridSize=(8, 8))
        img_clahe = clahe.apply(img_blurred)
        return img_clahe
    else:
        raise ValueError("Invalid method. Choose either 'clahe'.")

In [ ]:
def apply_morphological_closing(img, kernel_size=(5, 5)):
    # Create a kernel for morphological closing
    kernel = np.ones(kernel_size, np.uint8)

    # Apply morphological closing
    img_morphology = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

    return img_morphology

In [ ]:
def find_and_draw_contours(img_thresh):
    contours, _ = cv2.findContours(img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    img_contour = cv2.drawContours(img.copy(), contours, -1, (0,255,0), 3)

    return contours, len(contours), img_contour

In [ ]:
def filter_contours(contours, center, area_threshold, proximity_threshold):
    filtered_contours = []
    for contour in contours:
        if (cv2.contourArea(contour) > area_threshold and
            cv2.pointPolygonTest(contour, tuple(center), False) <= proximity_threshold):
            filtered_contours.append(contour)
    return filtered_contours

In [ ]:
def segment_image(img, contours):
    # Create an empty mask (single-channel)
    mask = np.zeros(img.shape[:2], dtype=np.uint8)

    # Draw filled contours on the mask
    cv2.drawContours(mask, contours, -1, (255), cv2.FILLED)

    # Apply the mask to the original image
    segmented_image = cv2.bitwise_and(img, img, mask=mask)

    segmented_image_uint8 = img_as_ubyte(segmented_image)

    return segmented_image_uint8


### Plot

In [ ]:
def plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours):

    axs[idx, 0].imshow(img_gray, cmap='gray')
    axs[idx, 0].axis('off')
    axs[idx, 0].set_title(f"Grayscale Image {idx+1}")

    axs[idx, 1].imshow(img_blurred, cmap='gray')
    axs[idx, 1].axis('off')
    axs[idx, 1].set_title(f"Blurred Image {idx+1}")

    axs[idx, 2].imshow(img_equalized, cmap='gray')
    axs[idx, 2].axis('off')
    axs[idx, 2].set_title(f"Equalized Image {idx+1}")

    axs[idx, 3].imshow(img_morphology, cmap='gray')
    axs[idx, 3].axis('off')
    axs[idx, 3].set_title(f"Morphology Image {idx+1}")

    axs[idx, 4].imshow(img_thresh, cmap='gray')
    axs[idx, 4].axis('off')
    axs[idx, 4].set_title(f"Thresh Image {idx+1}")

    axs[idx, 5].imshow(img_contour)
    axs[idx, 5].axis('off')
    axs[idx, 5].set_title(f"Contour Image {idx+1}\nNumber of contours: {num_contours}")

    axs[idx, 6].imshow(segmented_image.astype(np.uint8))
    axs[idx, 6].axis('off')
    axs[idx, 6].set_title(f"Segmented Image {idx+1}")

### Otsu Thresholding

In [ ]:
def otsu_thresh(img):
    # Convert to 8-bit unsigned integer
    img_uint8 = img_as_ubyte(img)

    # Apply Otsu's thresholding
    _, img_thresh = cv2.threshold(img_uint8, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Calculate Otsu's threshold value
    otsu_threshold_value = cv2.threshold(img_uint8, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]

    # Display the value
    print("Global Threshold Value:", otsu_threshold_value)

    return img_thresh

In [ ]:
# Load the first ten images
ten_images = images[:10]

# Create the figure and axes outside the loop
fig, axs = plt.subplots(10, 7, figsize=(20, 20))

for idx, img in enumerate(ten_images):
    img_gray, img_blurred = preprocess_image(img)
    img_equalized = contrast_enhancement(img_blurred, method='clahe', limit=0.01)
    img_morphology = apply_morphological_closing(img_equalized, kernel_size=(5, 5))
    img_thresh = otsu_thresh(img_morphology)
    contours, num_contours, img_contour = find_and_draw_contours(img_thresh)

    # filter contour
    center = (100, 100)  # center point
    area_threshold = 1000  # area threshold
    proximity_threshold = 100  # proximity threshold
    filtered_contours = filter_contours(contours, center, area_threshold, proximity_threshold)

    segmented_image = segment_image(img, filtered_contours)

    # Update the images in the existing axes
    plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Otsu Split 1/4

In [ ]:
def otsu_split_thresh(img):
    # Convert to 8-bit unsigned integer
    img_uint8 = img_as_ubyte(img)

    # Split the image into four vertical parts
    h, w = img_uint8.shape
    img_parts = [
        img_uint8[:, :w//4],
        img_uint8[:, w//4:w//2],
        img_uint8[:, w//2:3*w//4],
        img_uint8[:, 3*w//4:]
    ]

    # Apply Otsu's thresholding
    img_thresh_parts = []
    for part in img_parts:
        _, img_thresh = cv2.threshold(part, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Calculate Otsu's threshold value
        otsu_threshold_value = cv2.threshold(part, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]

        img_thresh_parts.append(img_thresh)

    # Concatenate the thresholded parts back into a single image
    img_thresh_combined = np.concatenate(img_thresh_parts, axis=1)

    return img_thresh_combined

In [ ]:
# Load the first ten images
ten_images = images[:10]

# Create the figure and axes outside the loop
fig, axs = plt.subplots(10, 7, figsize=(20, 20))

for idx, img in enumerate(ten_images):
    img_gray, img_blurred = preprocess_image(img)
    img_equalized = contrast_enhancement(img_blurred, method='clahe', limit=0.01)
    img_morphology = apply_morphological_closing(img_equalized, kernel_size=(5, 5))
    img_thresh = otsu_split_thresh(img_morphology)
    contours, num_contours, img_contour = find_and_draw_contours(img_thresh)

    # filter contour
    center = (100, 100)  # center point
    area_threshold = 1000  # area threshold
    proximity_threshold = 100  # proximity threshold
    filtered_contours = filter_contours(contours, center, area_threshold, proximity_threshold)

    segmented_image = segment_image(img, filtered_contours)

    # Update the images in the existing axes
    plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Otsu with NDI

In [ ]:
# Load the first ten images
ten_images = images[:10]

# Create the figure and axes outside the loop
fig, axs = plt.subplots(10, 7, figsize=(20, 20))

for idx, img in enumerate(ten_images):
    img_gray, img_blurred = preprocess_image_NDI(img)
    img_equalized = contrast_enhancement(img_blurred, method='clahe', limit=0.01)
    img_morphology = apply_morphological_closing(img_equalized, kernel_size=(5, 5))
    img_thresh = otsu_thresh(img_morphology)
    contours, num_contours, img_contour = find_and_draw_contours(img_thresh)

    # filter contour
    center = (100, 100)  # center point
    area_threshold = 1000  # area threshold
    proximity_threshold = 100  # proximity threshold
    filtered_contours = filter_contours(contours, center, area_threshold, proximity_threshold)

    segmented_image = segment_image(img, filtered_contours)

    # Update the images in the existing axes
    plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Adaptive Mean

In [ ]:
def adaptive_mean_thresh(img):
    # Convert to 8-bit unsigned integer
    img_uint8 = img_as_ubyte(img)

    # Apply adaptive thresholding (mean)
    img_thresh = cv2.adaptiveThreshold(img_uint8, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 199, 5)

    return img_thresh

In [ ]:
# Load the first ten images
ten_images = images[:10]

# Create the figure and axes outside the loop
fig, axs = plt.subplots(10, 7, figsize=(20, 20))

for idx, img in enumerate(ten_images):
    img_gray, img_blurred = preprocess_image(img)
    img_equalized = contrast_enhancement(img_blurred, method='clahe', limit=0.01)
    img_morphology = apply_morphological_closing(img_equalized, kernel_size=(5, 5))
    img_thresh = adaptive_mean_thresh(img_morphology)
    contours, num_contours, img_contour = find_and_draw_contours(img_thresh)

    segmented_image = segment_image(img, contours)

    # Update the images in the existing axes
    plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Adaptive Mean Split

In [ ]:
def adaptive_mean_split_thresh(img):
    # Convert to 8-bit unsigned integer
    img_uint8 = img_as_ubyte(img)

    # Split the image into four vertical parts
    h, w = img_uint8.shape
    img_parts = [
        img_uint8[:, :w//4],
        img_uint8[:, w//4:w//2],
        img_uint8[:, w//2:3*w//4],
        img_uint8[:, 3*w//4:]
    ]

    # Apply adaptive thresholding to each part
    img_thresh_parts = []
    for part in img_parts:
        img_thresh = cv2.adaptiveThreshold(part, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 199, 5)
        img_thresh_parts.append(img_thresh)

    # Concatenate the thresholded parts back into a single image
    img_thresh_combined = np.concatenate(img_thresh_parts, axis=1)

    return img_thresh_combined

In [ ]:
# Load the first ten images
ten_images = images[:10]

# Create the figure and axes outside the loop
fig, axs = plt.subplots(10, 7, figsize=(20, 20))

for idx, img in enumerate(ten_images):
    img_gray, img_blurred = preprocess_image(img)
    img_equalized = contrast_enhancement(img_blurred, method='clahe', limit=0.01)
    img_morphology = apply_morphological_closing(img_equalized, kernel_size=(5, 5))
    img_thresh = adaptive_mean_split_thresh(img_morphology)
    contours, num_contours, img_contour = find_and_draw_contours(img_thresh)

    segmented_image = segment_image(img, contours)

    # Update the images in the existing axes
    plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Adaptive Mean with NDI

In [ ]:
# Load the first ten images
ten_images = images[:10]

# Create the figure and axes outside the loop
fig, axs = plt.subplots(10, 7, figsize=(20, 20))

for idx, img in enumerate(ten_images):
    img_gray, img_blurred = preprocess_image_NDI(img)
    img_equalized = contrast_enhancement(img_blurred, method='clahe', limit=0.01)
    img_morphology = apply_morphological_closing(img_equalized, kernel_size=(5, 5))
    img_thresh = adaptive_mean_thresh(img_morphology)
    contours, num_contours, img_contour = find_and_draw_contours(img_thresh)

    segmented_image = segment_image(img, contours)

    # Update the images in the existing axes
    plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Adaptive Gaussian

In [ ]:
def adaptive_gaussian_thresh(img):
    # Convert to 8-bit unsigned integer
    img_uint8 = img_as_ubyte(img)

    # Apply adaptive thresholding (mean)
    img_thresh = cv2.adaptiveThreshold(img_uint8, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 299, 5)

    return img_thresh

In [ ]:
# Load the first ten images
ten_images = images[:10]

# Create the figure and axes outside the loop
fig, axs = plt.subplots(10, 7, figsize=(20, 20))

for idx, img in enumerate(ten_images):
    img_gray, img_blurred = preprocess_image(img)
    img_equalized = contrast_enhancement(img_blurred, method='clahe', limit=0.01)
    img_morphology = apply_morphological_closing(img_equalized, kernel_size=(5, 5))
    img_thresh = adaptive_gaussian_thresh(img_morphology)
    contours, num_contours, img_contour = find_and_draw_contours(img_thresh)

    segmented_image = segment_image(img, contours)

    # Update the images in the existing axes
    plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Adaptive Gaussian Split

In [ ]:
def adaptive_gaussian_split_thresh(img):
    # Convert to 8-bit unsigned integer
    img_uint8 = img_as_ubyte(img)

    # Split the image into four vertical parts
    h, w = img_uint8.shape
    img_parts = [
        img_uint8[:, :w//4],
        img_uint8[:, w//4:w//2],
        img_uint8[:, w//2:3*w//4],
        img_uint8[:, 3*w//4:]
    ]

    # Apply adaptive thresholding to each part
    img_thresh_parts = []
    for part in img_parts:
        img_thresh = cv2.adaptiveThreshold(part, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 199, 5)
        img_thresh_parts.append(img_thresh)

    # Concatenate the thresholded parts back into a single image
    img_thresh_combined = np.concatenate(img_thresh_parts, axis=1)

    return img_thresh_combined

In [ ]:
# Load the first ten images
ten_images = images[:10]

# Create the figure and axes outside the loop
fig, axs = plt.subplots(10, 7, figsize=(20, 20))

for idx, img in enumerate(ten_images):
    img_gray, img_blurred = preprocess_image(img)
    img_equalized = contrast_enhancement(img_blurred, method='clahe', limit=0.01)
    img_morphology = apply_morphological_closing(img_equalized, kernel_size=(5, 5))
    img_thresh = adaptive_gaussian_split_thresh(img_morphology)
    contours, num_contours, img_contour = find_and_draw_contours(img_thresh)

    segmented_image = segment_image(img, contours)

    # Update the images in the existing axes
    plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Adaptive Gaussian with NDI

In [ ]:
# Load the first ten images
ten_images = images[:10]

# Create the figure and axes outside the loop
fig, axs = plt.subplots(10, 7, figsize=(20, 20))

for idx, img in enumerate(ten_images):
    img_gray, img_blurred = preprocess_image_NDI(img)
    img_equalized = contrast_enhancement(img_blurred, method='clahe', limit=0.01)
    img_morphology = apply_morphological_closing(img_equalized, kernel_size=(5, 5))
    img_thresh = adaptive_gaussian_thresh(img_morphology)
    contours, num_contours, img_contour = find_and_draw_contours(img_thresh)

    segmented_image = segment_image(img, contours)

    # Update the images in the existing axes
    plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Rosin Threshold

In [ ]:
def rosin_threshold(image, k):
    hist, _ = np.histogram(image.flatten(), bins=256, range=(0, 256))

    threshold = 0
    done = False
    while not done:
        histogram_left = np.sum(hist[:threshold])
        histogram_right = np.sum(hist[threshold:])

        sum_left = np.sum(hist[:threshold] * np.arange(threshold))
        sum_right = np.sum(hist[threshold:] * np.arange(threshold, 256))

        mean_left = sum_left / (histogram_left + 1e-6)  # Avoid division by zero
        mean_right = sum_right / (histogram_right + 1e-6)  # Avoid division by zero

        threshold_next = int((mean_left + mean_right) / 2)

        if abs(threshold - threshold_next) <= k:
            done = True
        else:
            threshold = threshold_next

    _, thresholded_img = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)

    return thresholded_img

In [ ]:
# Load the first ten images
ten_images = images[:10]

# Create the figure and axes outside the loop
fig, axs = plt.subplots(10, 7, figsize=(20, 20))

for idx, img in enumerate(ten_images):
    img_gray, img_blurred = preprocess_image(img)
    img_equalized = contrast_enhancement(img_blurred, method='clahe', limit=0.01)
    img_morphology = apply_morphological_closing(img_equalized, kernel_size=(5, 5))
    img_morphology_uint8 = cv2.normalize(img_morphology, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    img_thresh = rosin_threshold(img_morphology_uint8, 5)

    contours, num_contours, img_contour = find_and_draw_contours(img_thresh)

    segmented_image = segment_image(img, contours)

    # Update the images in the existing axes
    plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Rosin Split 1/4

In [ ]:
def rosin_thresh_split(img, k):
    # Convert to 8-bit unsigned integer
    img_uint8 = img_as_ubyte(img)

    # Split the image into four vertical parts
    h, w = img_uint8.shape
    img_parts = [
        img_uint8[:, :w//4],
        img_uint8[:, w//4:w//2],
        img_uint8[:, w//2:3*w//4],
        img_uint8[:, 3*w//4:]
    ]

    # Apply adaptive thresholding to each part
    img_thresh_parts = []
    for part in img_parts:
        img_thresh = rosin_threshold(part, k)
        img_thresh_parts.append(img_thresh)

    # Concatenate the thresholded parts back into a single image
    img_thresh_combined = np.concatenate(img_thresh_parts, axis=1)

    return img_thresh_combined

In [ ]:
# Load the first ten images
ten_images = images[:10]

# Create the figure and axes outside the loop
fig, axs = plt.subplots(10, 7, figsize=(20, 20))

for idx, img in enumerate(ten_images):
    img_gray, img_blurred = preprocess_image(img)
    img_equalized = contrast_enhancement(img_blurred, method='clahe', limit=0.01)
    img_morphology = apply_morphological_closing(img_equalized, kernel_size=(5, 5))
    img_morphology_uint8 = cv2.normalize(img_morphology, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    img_thresh = rosin_thresh_split(img_morphology_uint8, 5)

    contours, num_contours, img_contour = find_and_draw_contours(img_thresh)

    segmented_image = segment_image(img, contours)

    # Update the images in the existing axes
    plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Rosin with NDI

In [ ]:
# Load the first ten images
ten_images = images[:10]

# Create the figure and axes outside the loop
fig, axs = plt.subplots(10, 7, figsize=(20, 20))

for idx, img in enumerate(ten_images):
    img_gray, img_blurred = preprocess_image_NDI(img)
    img_equalized = contrast_enhancement(img_blurred, method='clahe', limit=0.01)
    img_morphology = apply_morphological_closing(img_equalized, kernel_size=(5, 5))
    img_morphology_uint8 = cv2.normalize(img_morphology, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    img_thresh = rosin_threshold(img_morphology_uint8, 5)

    contours, num_contours, img_contour = find_and_draw_contours(img_thresh)

    segmented_image = segment_image(img, contours)

    # Update the images in the existing axes
    plot_images(axs, idx, img_gray, img_blurred, img_equalized, img_morphology, img_thresh, img_contour, segmented_image, num_contours)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.